In [7]:
from fastapi import FastAPI, Query
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


In [17]:
def recomendacion_juego(id_de_producto: int= Query(default='670290')):
    
    games = pd.read_parquet('Dataset/game_recommendation.parquet')

    id_de_producto = str(id_de_producto)
    games['id'] = games['id'].astype(str)

    atributos = games[games['id'] == id_de_producto]
    
    if atributos.empty:
        return 'No se encontró un juego con el ID proporcionado, prueba otro ID.'
    
    juegos = games[games['genres'] == atributos['genres'].iloc[0]]
    juegos= juegos[['title','genres']]
    juegos['caracteristicas'] = juegos[['genres']].apply(lambda row: '-'.join(row.values.astype(str)), axis=1) 
    juegos= juegos[['title','caracteristicas']].iloc[:1001,:]
    
    vector= TfidfVectorizer()
    matriz= vector.fit_transform(juegos['caracteristicas'])
    matriz_similaridad= cosine_similarity(matriz)
    
    indice_juego = juegos.index[juegos['title'] == atributos['title'].iloc[0]].tolist()[0]
    similares = list(enumerate(matriz_similaridad[indice_juego]))
    similaridades = sorted(similares, key=lambda x: x[1], reverse=True)[1:6]
    juegos_recomendados = [juegos.iloc[i[0]]['title'] for i in similaridades]
    
    juegos_recomendados

    return {'Juegos similares que te pueden interesar': juegos_recomendados}
recomendacion_juego("643980")

{'Juegos similares que te pueden interesar': ['SNOW - All Access Pro Pass',
  'SNOW - All Access Legend Pass',
  'SNOW - Starter Pack',
  'SNOW - Lifetime Pack',
  'RIFT']}